In [5]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [11]:
import os
os.environ['CURL_CA_BUNDLE'] = '1'

In [6]:
!pip install transformers
!pip install datasets
!pip install accelerate -U
!pip install transformers[torch]
!pip install sentencepiece

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [7]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorWithPadding
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, Dataset
from transformers import DataCollatorForSeq2Seq
from transformers import T5Tokenizer
from torch.utils.data import TensorDataset

In [8]:
# 查看GPU的信息
!nvidia-smi

Wed Nov  8 10:16:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.76       Driver Version: 515.76       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:8B:00.0 Off |                  N/A |
|  0%   27C    P8    26W / 350W |      2MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [14]:
# 加载微调所需的模型和tokenizer
model_name = "tscholak/2jrayxos"  # 替换为你的预训练模型的名称
tokenizer_name = "tscholak/2jrayxos"  # 替换为你的预训练模型的tokenizer名称
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
model_checkpoint = "tscholak/2jrayxos"
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, force_download=True, resume_download=False)

/root/miniconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.72.64.19'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/root/miniconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.72.64.19'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.72.64.19'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/

/root/miniconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.72.64.19'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.72.64.19'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


/root/miniconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.72.64.19'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.72.64.19'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.72.64.19'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/urllib3/connecti

/root/miniconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.72.64.19'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [16]:
# 初始化 T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained("tscholak/2jrayxos")
import json


# 指向数据集文件的路径
# dataset_path = "/content/drive/MyDrive/CSpider"

# 读取训练数据
with open("train.json", "r", encoding="utf-8") as train_file:
    train_data = json.load(train_file)

# 读取验证数据
with open("dev.json", "r", encoding="utf-8") as validation_file:
    validation_data = json.load(validation_file)

# 现在你可以使用 train_data 和 validation_data 进行进一步的处理，以适应你的项目需求。


/root/miniconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.72.64.19'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
# 修改预处理逻辑以适应CSpider数据集
def preprocess_cspider_data(data, tokenizer, max_source_length=128, max_target_length=64):
    formatted_data = []

    for example in data:
        question = example["question"]
        query = example["query"]

        # 根据模型输入的格式构建输入字符串
        input_string = f"question: {question} </s> sql: {query} </s>"

        # 使用 tokenizer 编码输入字符串
        input_encodings = tokenizer(
            input_string,
            truncation=True,
            padding="max_length",
            max_length=max_source_length,
            return_tensors="pt"
        )

        # 创建字典，包含输入、注意力掩码和目标
        formatted_data.append({
            "input_ids": input_encodings.input_ids[0],
            "attention_mask": input_encodings.attention_mask[0],
            "labels": tokenizer(query, truncation=True, padding="max_length", max_length=max_target_length, return_tensors="pt").input_ids[0],
        })

    return formatted_data

# 预处理训练数据
formatted_train_data = preprocess_cspider_data(train_data, tokenizer)

# 预处理验证数据
formatted_validation_data = preprocess_cspider_data(validation_data, tokenizer)

# 创建 Hugging Face Dataset
train_dataset = Dataset.from_dict({
    "input_ids": [item["input_ids"] for item in formatted_train_data],
    "attention_mask": [item["attention_mask"] for item in formatted_train_data],
    "labels": [item["labels"] for item in formatted_train_data]
}, split="train")

validation_dataset = Dataset.from_dict({
    "input_ids": [item["input_ids"] for item in formatted_validation_data],
    "attention_mask": [item["attention_mask"] for item in formatted_validation_data],
    "labels": [item["labels"] for item in formatted_validation_data]
}, split="validation")

print(train_dataset[1])
print(validation_dataset[1])

{'input_ids': [822, 10, 3, 2, 32101, 3, 87, 7, 3155, 11820, 40, 10, 3, 23143, 14196, 564, 3, 6, 2170, 834, 5540, 3, 6, 1246, 21680, 819, 4674, 11300, 272, 476, 1246, 32101, 3, 87, 7, 3155, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [3, 23143, 14196, 564, 3, 6, 2170, 834, 5540, 3, 6, 1246, 21680, 819, 4674, 11300, 272, 476, 1246, 1, 0, 0, 0, 0, 0, 0, 

In [18]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=2):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

show_random_elements(train_dataset)

,input_ids,attention_mask,labels
0,"[822, 10, 3, 2, 735, 2, 153, 735, 2, 153, 2, 58, 32101, 3, 87, 7, 3155, 11820, 40, 10, 3, 23143, 14196, 4505, 599, 382, 5411, 924, 106, 834, 715, 61, 21680, 622, 6577, 6157, 332, 536, 3, 15355, 3162, 16423, 6157, 332, 357, 9191, 332, 5411, 25697, 49, 834, 23, 26, 3274, 332, 4416, 25697, 49, 834, 23, 26, 549, 17444, 427, 332, 4416, 14672, 834, 4350, 3274, 96, 2, 121, 3430, 332, 4416, 5064, 834, 4350, 3274, 96, 2, 121, 117, 32101, 3, 87, 7, 3155, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[3, 23143, 14196, 4505, 599, 382, 5411, 924, 106, 834, 715, 61, 21680, 622, 6577, 6157, 332, 536, 3, 15355, 3162, 16423, 6157, 332, 357, 9191, 332, 5411, 25697, 49, 834, 23, 26, 3274, 332, 4416, 25697, 49, 834, 23, 26, 549, 17444, 427, 332, 4416, 14672, 834, 4350, 3274, 96, 2, 121, 3430, 332, 4416, 5064, 834, 4350, 3274, 96, 2, 121, 1]"
1,"[822, 10, 3, 2, 58, 32101, 3, 87, 7, 3155, 11820, 40, 10, 3, 23143, 14196, 3, 9, 208, 122, 599, 35, 52, 61, 21680, 1888, 32101, 3, 87, 7, 3155, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[3, 23143, 14196, 3, 9, 208, 122, 599, 35, 52, 61, 21680, 1888, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [19]:
batch_size = 3
args = Seq2SeqTrainingArguments(
    "test-summarization",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    # fp16=True,
)

In [20]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [21]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [22]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [23]:
trainer.train()

Epoch,Training Loss,Validation Loss


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 23.70 GiB total capacity; 21.61 GiB already allocated; 6.56 MiB free; 21.70 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# # 保存微调后的模型和分词器
# model.save_pretrained("my_finetuned_model")
# tokenizer.save_pretrained("my_finetuned_model")

In [ ]:
# # 从保存的文件中加载微调后的模型和分词器
# model = BertForSequenceClassification.from_pretrained("my_finetuned_model")
# tokenizer = BertTokenizer.from_pretrained("my_finetuned_model")

In [ ]:
!huggingface-cli login

In [ ]:
!sudo apt-get install git-lfs

In [ ]:
!git lfs install

In [ ]:
!huggingface-cli repo create tscholak/2jrayxos-CSpider-mrking_v2

In [ ]:
model.save_pretrained('tscholak/2jrayxos-CSpider-mrking_v2',push_to_hub=True)
tokenizer.save_pretrained('tscholak/2jrayxos-CSpider-mrking_v2',push_to_hub=True)